# 작물 병해 분류 AI 경진대회 Private 2위, Private Score: 0.99848,  

## yjunej팀, 사용모델: SE-ResNeXt101-32x4d, SE-ResNeXt26-32x4d

## requirements

### Pretrained model

* pretraining dataset: ImageNet
* SE-ResNeXt101-32x4d: ported from  https://cv.gluon.ai/model_zoo/classification.html#resnext, using timm library
* SE-ResNeXt26d_32x4d: https://rwightman.github.io/pytorch-image-models/models/resnext/, using timm library

## 라이브러리

In [15]:
import os
from glob import glob
from pytorch_lightning import callbacks
from pytorch_lightning.accelerators import accelerator
import torch
import torchvision
# from pytorch_lightning.plugins import DDPPlugin
from pytorch_lightning.strategies import DDPStrategy
from pytorch_lightning import seed_everything
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from torch import nn
import torch.nn.functional as F
import timm
from pytorch_lightning import LightningModule
from torchmetrics.functional import accuracy
from torchmetrics import F1Score
import numpy as np
import pandas as pd
from datetime import datetime
from torchvision import transforms
import os
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from pytorch_lightning import LightningDataModule
from sklearn.model_selection import StratifiedKFold
from torchvision.transforms.transforms import ColorJitter, RandomCrop, RandomHorizontalFlip

from config import Config
import cv2
import albumentations as A
import albumentations.pytorch as Ap

## EDA

In [16]:
# Data 폴더 상대 경로, ipynb 파일과 같은 depth에 존재하는 'fd_data' 폴더

DATA_DIR = './data'

train_df = pd.read_csv(os.path.join(DATA_DIR, 'train.csv'))
test_df = pd.read_csv(os.path.join(DATA_DIR, 'test.csv'))

print(train_df.head())
print(train_df.shape)
print(test_df.shape)

print(train_df['label'].value_counts())

              img_path  label description
0   ./train\가구수정\0.png      0        가구수정
1   ./train\가구수정\1.png      0        가구수정
2  ./train\가구수정\10.png      0        가구수정
3  ./train\가구수정\11.png      0        가구수정
4   ./train\가구수정\2.png      0        가구수정
(3457, 3)
(792, 2)
18    1405
10     595
1      307
3      210
15     162
2      145
11     142
7      130
6       99
9       57
5       54
17      51
14      27
12      22
13      17
4       14
0       12
16       5
8        3
Name: label, dtype: int64


* Train data가 매우 적고, imblanced class
* f1-macro 점수 향상을 위해서는 데이터가 적은 class에서도 좋은 점수를 얻어야합니다.

# Code

In [17]:
### Config & pre-defined function

# mixup augmentation을 위한 코드입니다.
def mixup_data(x, y, alpha=1.0):
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    batch_size = x.size()[0]
    index = torch.randperm(batch_size)
    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]

    return mixed_x, y_a, y_b, lam

def mixup_criterion(criterion, pred, y_a, y_b, lam, weight):
    return lam * criterion(pred, y_a, weight=weight) + (1 - lam) * criterion(pred, y_b,weight=weight)

class Config:
    exp = 'exp_1' # exp_1: 기존 train data만 사용, exp_2: pseudo labeling 추가한 최종 train data 사용
    phase = 'train' # train or test
    data_dir = './data' 
    model_name = "seresnext26d_32x4d"
    max_epochs = 25
    fold_num = 5 # k-fold 
    batch_size = 64 # 
    num_workers = 0
    seed = 41
    tta = True # Test Time Augmentation
    ckpt = None

## Pytorch lightning Dataset 

In [18]:
class FDDataset(Dataset):
    def __init__(self, cfg:Config, df:pd.DataFrame, aug:bool = True):
        super(FDDataset, self).__init__()
        self.cfg = cfg
        self.df = df
        self.aug = aug
        # Augmentation
        if self.aug:
            self.transform = transforms.Compose(
            [
                transforms.ToTensor(),
                transforms.Resize((256,256)),
                transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
                transforms.RandomHorizontalFlip(p=0.5),
                transforms.RandomVerticalFlip(p=0.5),
                transforms.RandomAffine(
                degrees=(-90,90),
                translate=(0.2, 0.2),
                scale=(0.8, 1.2), shear=15
                ),
            ]
            )
        else:
            self.transform = transforms.Compose(
            [
                transforms.ToTensor(),
                transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
                transforms.Resize((256,256)),
            ]
            )

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = os.path.join(self.cfg.data_dir, self.df.loc[idx, 'img_path'])
        
        img = cv2.imread(img_path)
        img = self.transform(img)
        if self.cfg.phase == 'test':
            return img, self.df.loc[idx, 'uid']
        label = self.df.loc[idx, 'label']
        return img, label
        
class FDDataModule(LightningDataModule):
    def __init__(self, cfg:Config):
        super().__init__()
        self.cfg = cfg
        self.test_df = pd.read_csv(os.path.join(cfg.data_dir, 'test.csv'))
        
        # exp_1은 train.csv 데이터 사용
        if 'exp_1'  in self.cfg.exp:
            self.train_df = pd.read_csv(os.path.join(cfg.data_dir, 'train.csv'))
            print('Data: train.csv')
        # exp 2는 pseudo-labeling data를 추가한 full_train.csv 파일을 load
        elif 'exp_2' in self.cfg.exp:
            self.train_df = pd.read_csv(os.path.join(cfg.data_dir, 'full_train.csv'))
            print('Data: full_train.csv')
        self.fold_num = 0
        self._split_kfold()

    def set_fold_num(self, fold_num):
        self.fold_num = fold_num
    
    # weighted crossentropy loss를 위한 weight 계산 함수
    def get_class_weight(self):
        return 1 / self.train_df['label'].value_counts().sort_index().values

    def setup(self, stage=None):
        if stage != 'test':
            print(f'FOLD NUM:{self.fold_num}')
            train_df = self.train_df[
                self.train_df["kfold"] != self.fold_num
            ].reset_index(drop=True)
            val_df = self.train_df[
                self.train_df["kfold"] == self.fold_num
            ].reset_index(drop=True)

            self.train = FDDataset(self.cfg, train_df, aug=True)
            self.val = FDDataset(self.cfg, val_df, aug=False)
            self.train_fold_df = self.train_df

        if stage == 'test':
            self.test = FDDataset(self.cfg, self.test_df, aug=False)

    # Stratified KFold 
    def _split_kfold(self):
        skf = StratifiedKFold(
            n_splits=Config.fold_num, shuffle=True, random_state=Config.seed
        )
        # (train_idx, val_idx)
        for n, (_, val_index) in enumerate(
            skf.split(
                X=self.train_df,
                y=self.train_df['label']
            )
        ):  # if valid index, record fold num in 'kfold' column
            self.train_df.loc[val_index, "kfold"] = int(n)

    def train_dataloader(self):
        return DataLoader(
            self.train,
            batch_size=self.cfg.batch_size,
            num_workers=self.cfg.num_workers,
            shuffle=True,
            pin_memory=True,
        )
    
    def val_dataloader(self):
        return DataLoader(
            self.val,
            batch_size=self.cfg.batch_size,
            num_workers=self.cfg.num_workers,
            shuffle=False,
            pin_memory=True,
        )
    
    def test_dataloader(self):
        return DataLoader(
            self.test,
            batch_size=self.cfg.batch_size,
            num_workers=self.cfg.num_workers,
            shuffle=False,
        )


## Pytorch lightning Module

In [19]:
class FDModel(nn.Module):
    def __init__(self, cfg:Config):
        super(FDModel, self).__init__()
        self.cfg = cfg
        self.cnn = timm.create_model( # timm ImageNet pre-trained 모델 load
            cfg.model_name,
            pretrained=True,
            num_classes = 7,
            in_chans = 3
        )
        
    def forward(self, x):
        out = self.cnn(x)
        return out

class FDModule(LightningModule):
    def __init__(self, cfg:Config, class_weight=None):
        super().__init__()
        self.model = FDModel(cfg)
        self.val_metric = F1Score(task='multiclass', num_classes=19, average="weighted")
        self.train_metric =  F1Score(task='multiclass', num_classes=19, average="weighted")
        self.cfg = cfg
        self.lr = 1e-4

        self.class_weight = class_weight
        
        self.softmax = torch.nn.Softmax(dim=1)
        
        ## TTA에 사용할 augmentation
        self.horizontalflip = transforms.RandomHorizontalFlip(p=1)
        self.verticalflip = transforms.RandomVerticalFlip(p=1)
        self.rotation_left = transforms.RandomRotation(degrees=(-90,-90))
        self.rotation_right = transforms.RandomRotation(degrees=(90,90))

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        if batch_idx % 4 == 0: # 4 step 주기로 mixup 사용
            mixed_x, y_a, y_b, lam = mixup_data(x, y)
            logits = self(mixed_x)
            loss = mixup_criterion(F.cross_entropy, logits, y_a, y_b, lam, torch.Tensor(self.class_weight).cuda())
            self.log_dict({'mixup_loss':loss})
            return loss
        logits = self(x)
        loss = F.cross_entropy(logits, y.long(), weight= torch.Tensor(self.class_weight).cuda())
        preds = torch.argmax(logits, dim=1)
        micro_acc = accuracy(preds, y)
        f1_score = self.train_metric(preds, y)
        self.log_dict(
            {
                "train_loss": loss,
                "train_acc": micro_acc,
                "train_f1_macro": f1_score
            },
            prog_bar=True,
            sync_dist=True,
            on_step=False,
            on_epoch=True

        )
        return loss
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y.long(), weight= torch.Tensor(self.class_weight).cuda())
        preds = torch.argmax(logits, dim=1)
        micro_acc = accuracy(preds, y)

        f1_score = self.val_metric(preds, y)
        self.log_dict(
            {
                "val_loss": loss,
                "val_acc": micro_acc,
                "val_f1_macro": f1_score                
            },
            prog_bar=True,
            sync_dist=True,
            on_step=False,
            on_epoch=True
        )

    def test_step(self, batch, batch_idx):
        if self.cfg.tta:
            return self.tta(batch,batch_idx)
        x, uid = batch
        logits = self(x)
        prob = self.softmax(logits)
        preds = prob

        return preds, uid

    def tta(self, batch, batch_idx):
        x, uid = batch
        _normal = self.softmax(self(x))
        _h_flip = self.softmax(self(self.horizontalflip(x)))
        _v_flip = self.softmax(self(self.verticalflip(x)))
        _l_rotate = self.softmax(self(self.rotation_left(x)))
        _r_rotate = self.softmax(self(self.rotation_right(x)))
        preds = (_normal + _h_flip + _v_flip + _l_rotate + _r_rotate) / 5
        return preds, uid  

    def test_epoch_end(self, outputs):
        results = self.all_gather(outputs)
        
        # class 별 confidence 저장하는 dataframe
        df = pd.DataFrame(range(20000,24750),columns=['uid'])
        df['prob_0'] = -100.0
        df['prob_1'] = -100.0
        df['prob_2'] = -100.0
        df['prob_3'] = -100.0
        df['prob_4'] = -100.0
        df['prob_5'] = -100.0
        df['prob_6'] = -100.0
        
        df = df.set_index('uid')
        for p, u in results:
            prob = p.reshape(-1,7).cpu().numpy()
            u = u.reshape(-1).cpu().numpy()
            for pp, uu in zip(prob,u):
                df.loc[uu] = pp
        df.to_csv(f'result_{self.cfg.exp}.csv')

    def configure_optimizers(self):
        if 'exp_1' in self.cfg.exp:
            optimizer = torch.optim.AdamW(self.parameters(), lr=self.lr)
            print('optimizer: AdamW')
            if 'exp_1_1' in self.cfg.exp:
                scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=200, T_mult=1)
                print('scheduler: CosineAnnealing')
            elif 'exp_1_2' in self.cfg.exp:
                scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=40, verbose=True)
                print('scheduler: ReduceLROnPlateau')
                return (
                {
                "optimizer":optimizer,
                "lr_scheduler": {"scheduler":scheduler, "monitor":"val_loss", "interval":"epoch"},
                },
                )
        elif 'exp_2_2' in self.cfg.exp:
            optimizer = torch.optim.AdamW(self.parameters(), lr=self.lr)
            print('optimizer: AdamW')
            scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=100, T_mult=1)
            print('scheduler: CosineAnnealing')
            return (
            {
                "optimizer":optimizer,
                "lr_scheduler": {"scheduler":scheduler, "monitor":"val_loss", "interval":"epoch"},
            },
            )
        
        
        elif 'exp_2_1' in self.cfg.exp:
            optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
            print('optimizer: Adam')
            scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=100, T_mult=1)
            print('scheduler: CosineAnnealing')
            return (
            {
                "optimizer":optimizer,
                "lr_scheduler": {"scheduler":scheduler, "monitor":"val_loss", "interval":"epoch"},
            },
            )
        
        return [optimizer], [scheduler]
        
    

In [20]:
import os
from glob import glob
from pytorch_lightning import callbacks
from pytorch_lightning.accelerators import accelerator

# from pytorch_lightning.plugins import DDPPlugin
from pytorch_lightning.strategies import DDPStrategy
from pytorch_lightning import seed_everything
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping


def train(cfg: Config, fold_num):
    seed_everything(Config.seed)
    gpus = torch.cuda.device_count()
    fd_data_module = FDDataModule(cfg)
    fd_data_module.set_fold_num(fold_num)
    fd_data_module.setup()
    class_weight = fd_data_module.get_class_weight()

    if cfg.phase=='test':
        fd_module = FDModule(cfg, class_weight=None).load_from_checkpoint(cfg.ckpt,
         cfg=Config)
    else:
        fd_module = FDModule(cfg, class_weight=class_weight)
    

    model_checkpoint = ModelCheckpoint(monitor='val_loss', save_top_k=5, dirpath=f'results/{cfg.exp}/{fd_data_module.fold_num}',
    filename="{epoch:02d}-{val_loss:.6f}-{val_acc:.4f}-{val_f1_macro:.6f}.pth", mode='min')
    trainer = pl.Trainer(
#         gpus=0,
        accelerator='gpu',
        num_nodes=1,
        deterministic=True,
        check_val_every_n_epoch=1,
        callbacks = [model_checkpoint],
        precision=16,
        log_every_n_steps=4,
        max_epochs = cfg.max_epochs,
#         auto_lr_find=True,
#         strategy = DDPStrategy(find_unused_parameters=False),
    )
    
    if cfg.phase == 'train':
        trainer.fit(fd_module, fd_data_module)
    else:
        trainer.test(fd_module, fd_data_module)

In [21]:
Config.exp = 'exp_1_1'
train(Config, 0)
train(Config, 1)
train(Config, 2)
train(Config, 3)
train(Config, 4)

Global seed set to 41


Data: train.csv
FOLD NUM:0


C:\Users\user\anaconda3\envs\venv\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\user\anaconda3\envs\venv\lib\site-packages\lightning_fabric\connector.py:562: UserWarning: 16 is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
  rank_zero_warn(
Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOC

FOLD NUM:0
optimizer: AdamW
scheduler: CosineAnnealing


Sanity Checking: 0it [00:00, ?it/s]

C:\Users\user\anaconda3\envs\venv\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:430: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


TypeError: pic should be PIL Image or ndarray. Got <class 'NoneType'>

In [ ]:
Config.exp = 'exp_1_2'
Config.model_name = 'gluon_seresnext101_32x4d'

train(Config, 0)
train(Config, 1)
train(Config, 2)
train(Config, 3)
train(Config, 4)


In [ ]:
exp_1_1_ckpt = []
exp_1_2_ckpt = []


for f in range(5):
    fold_lst = glob(f'results/exp_1_1/{f}/*')
    _min = 10000
    idx = 0
    for i, v in enumerate([float(x.split('val_loss=')[1].split('-')[0]) for x in fold_lst]):
        if v < _min:
            _min = v
            idx = i
    exp_1_1_ckpt.append(fold_lst[idx])

for f in range(5):
    fold_lst = glob(f'results/exp_1_2/{f}/*')
    _min = 10000
    idx = 0
    for i, v in enumerate([float(x.split('val_loss=')[1].split('-')[0]) for x in fold_lst]):
        if v < _min:
            _min = v
            idx = i
    exp_1_2_ckpt.append(fold_lst[idx])


In [ ]:
exp_1_ckpt = []

for i, (e_1, e_2) in enumerate(zip(exp_1_1_ckpt, exp_1_2_ckpt)):
    ckpt = e_1 if float(e_1.split('val_loss=')[1].split('-')[0]) <= float(e_2.split('val_loss=')[1].split('-')[0]) else e_2
    exp_1_ckpt.append(ckpt)

In [ ]:
exp_1_1_ckpt

In [ ]:
exp_1_2_ckpt

In [ ]:
exp_1_ckpt

### Inference

In [ ]:
Config.phase = 'test'

def test(cfg: Config):
    seed_everything(Config.seed)
    fd_data_module = FDDataModule(cfg)
    fd_data_module.setup(stage='test')
    fd_module = FDModule(cfg, class_weight=None).load_from_checkpoint(cfg.ckpt, cfg=Config)
    
    
    model_checkpoint = ModelCheckpoint(monitor='val_loss', save_top_k=3, dirpath=f'results/{cfg.exp}/{fd_data_module.fold_num}',
    filename="{epoch:02d}-{val_loss:.6f}-{val_acc:.4f}-{val_f1_macro}.pth", mode='min')
    
    trainer = pl.Trainer(
#         gpus="0",
        accelerator='gpu',
        num_nodes=1,
        deterministic=True,
        check_val_every_n_epoch=1,
        callbacks = [model_checkpoint],
        precision=16,
        log_every_n_steps=4,
        max_epochs = 1,
#         auto_lr_find=True,
#         strategy = DDPStrategy(find_unused_parameters=False),
    )
    trainer.test(fd_module, fd_data_module)

In [ ]:
for i,ckpt in enumerate(exp_1_ckpt):
    Config.exp = f'exp_1_fold_{i}'
    Config.ckpt = ckpt
    Config.model_name = 'gluon_seresnext101_32x4d' if 'exp_1_2' in Config.ckpt else "seresnext26d_32x4d"
    test(Config)

## Ensemble, Psuedo Labeling

In [ ]:
fold_0 = pd.read_csv('result_exp_1_fold_0.csv')
fold_1 = pd.read_csv('result_exp_1_fold_1.csv')
fold_2 = pd.read_csv('result_exp_1_fold_2.csv')
fold_3 = pd.read_csv('result_exp_1_fold_3.csv')
fold_4 = pd.read_csv('result_exp_1_fold_4.csv')

df = pd.concat([fold_0,fold_1,fold_2,fold_3,fold_4])
df = df.groupby('uid').mean()
df['_max'] = df.max(axis=1)
df['label'] = df.idxmax(axis=1).str[-1].astype(int)
pseudo_label_df = df[df['_max'] > 0.85][['label']] # 4211 len
pseudo_label_df['img_path'] = 'test_imgs/'+pseudo_label_df.index.astype(str)+'.jpg'
pseudo_label_df = pseudo_label_df.reset_index()[['uid','img_path','label']]
org_train = pd.read_csv('./data/train.csv')

full_train_df = pd.concat([org_train[['uid','img_path','label']], pseudo_label_df],axis=0)
full_train_df.to_csv('./data/full_train.csv',index=False)

# Training

In [ ]:
Config.exp = 'exp_2_1'
Config.model_name = 'gluon_seresnext101_32x4d'
Config.phase = 'train'
Config.max_epochs=300

In [ ]:
train(Config, 0)
train(Config, 1)
train(Config, 2)
train(Config, 3)
train(Config, 4)

In [ ]:
Config.exp = 'exp_2_2'

In [ ]:
train(Config, 0)
train(Config, 1)
train(Config, 2)
train(Config, 3)
train(Config, 4)

In [ ]:
exp_2_1_ckpt = []
exp_2_2_ckpt = []


for f in range(5):
    fold_lst = glob(f'results/exp_2_1/{f}/*')
    _min = 10000
    idx = 0
    for i, v in enumerate([float(x.split('val_loss=')[1].split('-')[0]) for x in fold_lst]):
        if v < _min:
            _min = v
            idx = i
    exp_2_1_ckpt.append(fold_lst[idx])

for f in range(5):
    fold_lst = glob(f'results/exp_2_2/{f}/*')
    _min = 10000
    idx = 0
    for i, v in enumerate([float(x.split('val_loss=')[1].split('-')[0]) for x in fold_lst]):
        if v < _min:
            _min = v
            idx = i
    exp_2_2_ckpt.append(fold_lst[idx])

In [ ]:
exp_2_ckpt = []

for i,(e_1, e_2) in enumerate(zip(exp_2_1_ckpt, exp_2_2_ckpt)):
    ckpt = e_1 if float(e_1.split('val_loss=')[1].split('-')[0]) <= float(e_2.split('val_loss=')[1].split('-')[0]) else e_2
    exp_2_ckpt.append(ckpt) 

In [ ]:
exp_2_1_ckpt

In [ ]:
exp_2_2_ckpt

In [ ]:
exp_2_ckpt

### Inference

In [ ]:
Config.phase = 'test'

for i,ckpt in enumerate(exp_2_ckpt):
    Config.exp = f'exp_2_fold_{i}'
    Config.ckpt = ckpt
    test(Config)

## Submit

In [ ]:
fold_0 = pd.read_csv('result_exp_2_fold_0.csv')
fold_1 = pd.read_csv('result_exp_2_fold_1.csv')
fold_2 = pd.read_csv('result_exp_2_fold_2.csv')
fold_3 = pd.read_csv('result_exp_2_fold_3.csv')
fold_4 = pd.read_csv('result_exp_2_fold_4.csv')

df = pd.concat([fold_0,fold_1,fold_2,fold_3,fold_4])
df = df.groupby('uid').mean()
series = df.idxmax(axis=1).str[-1].astype(int)
result = pd.DataFrame(series,columns=['label'])
result.to_csv('submit.csv')

In [ ]:
pd.read_csv('submit.csv').head()